# 线性规划 Linear Programming
参考自《Python与运筹优化》

## 逻辑的数学表示
### 条件约束
$ B>0 \space if \space A>0$ 等价于
$$
s.t.
    \begin{cases}
    s \cdot A - max(A, 0) = 0 \\
    s \cdot M + B > 0 \\
    s = 0 \space or \space 1
    \end{cases}
$$ 
其中M为一个充分大的数

- 或逻辑：

$ B>0 \space if \space A_1>0 \space or \space A_2>0$ 等价于
$$
s.t.
    \begin{cases}
    s_1 \cdot A_1 - max(A, 0) = 0 \\
    s_2 \cdot A_2 - max(A, 0) = 0 \\
    (s_1 + s_2) \cdot M + B > 0 \\
    s_1, s_2 = 0 \space or \space 1
    \end{cases}
$$ 

- 与逻辑：

$ B>0 \space if \space A_1>0 \space and \space A_2>0$ 等价于
$$
s.t.
    \begin{cases}
    s_1 \cdot A_1 - max(A, 0) = 0 \\
    s_2 \cdot A_2 - max(A, 0) = 0 \\
    (|s_1 + s_2| - |s_1 - s_2|) \cdot M + B > 0 \\
    s_1, s_2 = 0 \space or \space 1
    \end{cases}
$$ 

## 运输问题

### 产销平衡且确定

In [13]:
import os
os.chdir('..')  # 改变工作路径

from rich import print
import numpy as np
from problem_solving.transportation_problem import balance_transportation_problem
##  示例
c = np.array([[3, 11, 3, 10],  # c[i, j] 为从A_i到B_j的单位运费
              [1, 9, 2, 8],
              [7, 4, 10, 5]])
sales_volume = np.array([3, 6, 5, 6]) # B_j销量 j = 0, 1, 2, 3
production = np.array([7, 4, 9])  # A_i产量

print(*balance_transportation_problem(c, production, sales_volume), sep='\n\n')

[[2. 0. 5. 0.]
 [1. 0. 0. 3.]
 [0. 6. 0. 3.]]

85.0

### 产销不平衡
- 修改线性规划代码
- 转换成平衡问题, 虚拟出一个垃圾倾销地, 或者一个新的产地

这里展示将问题转换成平衡问题

In [16]:
# 供过于求
production2 = production + 1
print(production2)

c2 = np.hstack((c, np.array([0, 0, 0]).reshape(3, 1)))
print(c2)

sales_volume2 = np.hstack((sales_volume, [np.sum(production2) - np.sum(sales_volume)]))
print(sales_volume2)

print(*balance_transportation_problem(c2, production2, sales_volume2), sep='\n\n')

[ 8  5 10]

[[ 3 11  3 10  0]
 [ 1  9  2  8  0]
 [ 7  4 10  5  0]]

[3 6 5 6 3]

[[0. 0. 5. 0. 3.]
 [3. 0. 0. 2. 0.]
 [0. 6. 0. 4. 0.]]

78.0

### 产销不确定

这里演示如何转换成平衡问题

In [35]:
Num = float | int

class RangeNum(object):
    def __init__(self, l) -> None:
        super().__init__()
        if isinstance(l, tuple):
            assert l[0] <= l[1]
            if l[0] == l[1]:
                self.lower_bound = self.upper_bound = l[0]
            else:
                self.lower_bound, self.upper_bound = l
            return
        assert isinstance(l, Num)
        self.lower_bound = self.upper_bound = l
    
    def is_no_range(self):
        return self.lower_bound == self.upper_bound
    
    def __str__(self) -> str:
        if self.is_no_range():
            return str(self.lower_bound)
        return f'{self.lower_bound}~{self.upper_bound}'

    def __repr__(self) -> str:
        return str(self)

sales_volume3 = np.array([RangeNum(i) for i in [3, (4, 7), 5, 6]])
print(sales_volume3)
M = 1000
c3 = c = np.array([[3, 11, 3, 10, 11],  # c[i, j] 为从A_i到B_j的单位运费
                   [1, 9, 2, 8, 9],
                   [7, 4, 10, 5, 4],
                   [M, M, M, M, 0]])
sales_volume3 = np.array([3, 4, 5, 6, 7-4])
production3 = np.array([7, 4, 9, 1])

print(*balance_transportation_problem(c3, production3, sales_volume3), sep='\n\n')

[3 4~7 5 6]

[[2. 0. 5. 0. 0.]
 [1. 0. 0. 3. 0.]
 [0. 4. 0. 3. 2.]
 [0. 0. 0. 0. 1.]]

85.0

这里的结果与第一例一样, 因为肯定不做无用功, 如果是有两个不确定, 这个例子就显得更有意义些

## 灵敏度分析
全部摘抄自《Python与运筹优化》

In [17]:
import pandas as pd
import gurobipy as gp

def LP_Model_Analysis(MODEL,precision=3):
    if MODEL.status == gp.GRB.Status.OPTIMAL:
        pd.set_option('display.precision', precision)
        #设置精度
        print("\nGlobal optimal solution found.")
        print(f"Objective Sense:{'MINIMIZE' if MODEL.ModelSense ==1 else 'MAXIMIZE'}")
        print(f"Objective Value = {MODEL.ObjVal}")
        try:
            display(pd.DataFrame([[var.X, var.RC]for var in MODEL.getVars()],
                               index=[var.Varname for var in MODEL.getVars()],
                               columns=["Value", "Reduced Cost"]))
            print('左边是求解出的变量取值, 右边表示减少一个单位这个变量的将增加多少代价',
                  '因为变量被默认限制大于等于0, 所以可以看到x[1]和x[6]已经减至最低', sep='\n')
            display(pd.DataFrame([[Constr.Slack, Constr.pi] for Constr in MODEL.getConstrs()],
                               index=[Constr.constrName for Constr in MODEL.getConstrs()],
                               columns=["Slack or Surplus", "DualPrice"]))
            print('这个表格说明了约束条件右侧系数对解的影响',
                  '中间一栏是松弛变量, 就是约束条件右侧减左侧还剩多少, 可以理解为资源还剩多少, 等式约束显得没有意义',
                  '右边一栏是影子价格, 表明目标对右侧系数(这种资源)的变化率, 具体在表格中就说明了增加一桶牛奶会带来37.92的收益',
                  '这是求最大目标时的情况, 求最小时会不会反过来还没试过', sep='\n')
            print("\nRanges in which the basis is unchanged:")
            display(pd.DataFrame([[var.Obj,var.SAObjLow,var.SAObjUp]for var in MODEL.getVars()],
                               index=[var.Varname for var in MODEL.getVars()],
                               columns=["Cofficient","Allowable Minimize","Allowable Maximize"]))
            print('决策变量价值系数在什么范围内变动不会使最优解(决策变量取值)改变',
                  '应该指的是控制其他不变只有这个变时的情况', sep='\n')
            print("Righthand Side Ranges:")
            display(pd.DataFrame([[Constr.RHS, Constr.SARHSLow, Constr.SARHSUp] for Constr in MODEL.getConstrs()], 
                               index=[Constr.constrName for Constr in MODEL.getConstrs()],
                               columns=["RHS","Allowable Minimize","Allowable Maximize"]))
        except:
            display(pd.DataFrame([var.X for var in MODEL.getVars()],
                               index=[var.Varname for var in MODEL.getVars()],
                               columns=["Value"]))
            display(pd.DataFrame([Constr.Slack for Constr in MODEL.getConstrs()],
                               index=[Constr.constrName for Constr in MODEL.getConstrs()],
                               columns=["Slack or Surplus"]))

In [18]:
model = gp.Model()
x = model.addVars(range(1, 7), name='x')
model.update()
Const = [24, 16, 44, 32, -3, -3]
model.setObjective(sum(x[i+1] * Const[i] for i in range(6)), sense=gp.GRB.MAXIMIZE)

model.addConstr(1/3 * (x[1]+x[5]) + 1/4 * (x[2]+x[6]) <= 50, name='Milk')
model.addConstr(4 * (x[1]+x[5]) + 2 * (x[2]+x[6]) + 2 * (x[5]+x[6]) <= 480, name='Time')
model.addConstr(x[1] + x[5] <= 100, name='CPCT')
model.addConstr(x[3] - 0.8 * x[5] == 0)
model.addConstr(x[4] - 3/4 * x[6] == 0)
# x默认大于等于0
model.optimize()
LP_Model_Analysis(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 14 nonzeros
Model fingerprint: 0xff031ad2
Coefficient statistics:
  Matrix range     [3e-01, 6e+00]
  Objective range  [3e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 5e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8300000e+03   7.370950e+01   0.000000e+00      0s
       3    3.4608000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.460800000e+03


Global optimal solution found.

Objective Sense:MAXIMIZE

Objective Value = 3460.8

,Value,Reduced Cost
x[1],0.0,-1.68
x[2],168.0,0.00
x[3],19.2,0.00
x[4],0.0,0.00
x[5],24.0,0.00
x[6],0.0,-1.52


...

Ranges in which the basis is unchanged:

,Cofficient,Allowable Minimize,Allowable Maximize
x[1],24.0,-inf,25.680
x[2],16.0,13.900,24.150
x[3],44.0,40.833,63.750
x[4],32.0,-inf,34.027
x[5],-3.0,-5.533,12.800
x[6],-3.0,-inf,-1.480


决策变量价值系数在什么范围内变动不会使最优解(决策变量取值)改变
应该指的是控制其他不变只有这个变时的情况

Righthand Side Ranges:

,RHS,Allowable Minimize,Allowable Maximize
Milk,50.0,26.667,60.000
Time,480.0,400.000,733.333
CPCT,100.0,24.000,inf
R3,0.0,-19.200,inf
R4,0.0,0.000,inf
